# Geospatial mapping with Python
Shahryar Noei

Originally by Marco Chierici

May 13, 2025

---

# Overview

In this lab, we'll go through three main methods to deal with geospatial mapping in Python, with a focus on choropleth maps.

Prerequisites:

* GeoPandas & geoplot - `conda install -c conda-forge geopandas`, `conda install -c conda-forge geoplot`
* Plotly - `conda install -c plotly plotly=5.22.0`
* Folium - `conda install -c conda-forge folium`

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# don't display warning messages
warnings.filterwarnings(action="ignore")

%matplotlib inline

# Ingredients

A map is a set of polygon coordinates displayed on a 2D canvas. So, first you need to get those polygons! Common sources are:

- a shapefile (shp) or GeoJSON file - `geopandas` is the preferred way to import these files into Python
- a Python library (`geopandas` again, or `basemap`) with the information for common locations or regions (Europe, USA, etc.)
- OpenStreetMap or other similar tile providers.

Now that you have a set of polygons, you can plot it using different approaches. For example:

- `geoplot`, if you have your data into a `geopandas`' `geodataframe`;
- `plotly`, if you want or need an interactive map from a `geodataframe`;
- `folium`, if you want to use Google Map styled maps.

## Choropleth maps

A choropleth map is a map composed of colored polygons, conveying spatial variations of a quantity. To create them, you'll need:

1. Geometry information (GeoJSON, SHP, etc.) - see above
1. A list of values indexed by a feature identifier.

# GeoPandas

[GeoPandas](https://geopandas.org/en/stable/) extends the datatypes used by pandas to support spatial operations on geometric types.

We start by importing a geoJSON file with the state boundaries of France.

In [ ]:
# installation
# !pip install geopandas
# !conda install geopandas
# !mamba install geopandas

In [ ]:
import geopandas as gpd

data = gpd.read_file("https://raw.githubusercontent.com/holtzy/The-Python-Graph-Gallery/master/static/data/france.geojson")
data.crs

`data` is a `geodataframe`, where each row represents an item in the geoJSON file - in this case, a region of France. The columns describe the features of each region: in particular, the `geometry` column stores the coordinates of the region boundary.

In [ ]:
print(type(data))

The most common and straightforward way to plot a map from a geopandas dataframe is with `geoplot`.

In [ ]:
import geoplot as gplt
import geoplot.crs as gcrs  # load projections

gplt.polyplot(
    data,
    projection=gcrs.AlbersEqualArea(),
    edgecolor="darkgrey",
    facecolor="lightgrey",
    linewidth=0.3,
    figsize=(12, 8),
)
plt.show()

Now that we have learned how to draw a map combining GeoPandas and geoplot, let's create a choropleth map to visualize the unemployment rate of each US county.

In [ ]:
# Load the json file with county coordinates
geoData = gpd.read_file("https://raw.githubusercontent.com/holtzy/The-Python-Graph-Gallery/master/static/data/US-counties.geojson")

# Make sure the "id" column is an integer
geoData.id = geoData.id.astype(str).astype(int)
geoData.head()

In [ ]:
# Remove Alaska, Hawaii and Puerto Rico.
stateToRemove = ["02", "15", "72"]
geoData = geoData[~geoData.STATE.isin(stateToRemove)]
geoData = geoData.explode(index_parts=False)

# Basic plot with just county outlines
gplt.polyplot(geoData, figsize=(20, 4))

plt.show()

To build a choropleth map, we need numeric data to link to the map: we load the unemployment rate of US counties from the Bureau of Labor Statistics ([source](http://www.bls.gov/lau/#tables)).

In [ ]:
data = pd.read_csv("https://raw.githubusercontent.com/holtzy/The-Python-Graph-Gallery/master/static/data/unemployment-x.csv")
data.head()

In [ ]:
# plot a histogram of unemployment rate
sns.distplot(data["rate"], hist=True, kde=False);

Now we can merge both sources of data (map and unemployment rates) in order to visualize them together. We'll be using  the `merge()` function of geopandas:

In [ ]:
fullData = geoData.merge(data, left_on=["id"], right_on=["id"])
fullData.head()

We are happy that `geoplot` provides a `choropleth()` function!

`choropleth(df, projection=None, hue=None, cmap=None, scheme=None)`

The `hue` parameter expects the name of the column we want to use to control the color of each county. Then, we have to pick a suitable color palette (`cmap`) and a binning scheme (`scheme`). Geoplot comes with both continuous and categorical binning schemes, i.e. methods that split a sequence of observations into a number of bins.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 12))

# set up the binning sheme
import mapclassify as mc

scheme = mc.Quantiles(fullData["rate"], k=10)

# map
gplt.choropleth(
    fullData,
    hue="rate",
    linewidth=0.1,
    scheme=scheme,#necessary in order to make sure that we limit the number of colors
    cmap="inferno_r",
    legend=True,
    edgecolor="black",
    ax=ax,
)

ax.set_title("Unemployment rate in US counties", fontsize=13);

# Plotly

[Plotly](https://plotly.com/graphing-libraries/) is a open-source graphing library aimed at interactivity for Python, R, and other languages. It can be used offline and does not require any account registration (Plotly.com has also paid licenses for enterprise users).

We'll create a choropleth map of the US unemployment rate with Plotly, this time using a slightly different approach for gathering the data.

We load a GeoJSON file with the geometry information for US counties:

In [ ]:
# installation
# !pip install plotly==5.22.0
# !conda install -y -c plotly plotly=5.22.0
# !mamba install -y -c plotly plotly=5.22.0

In [ ]:
from urllib.request import urlopen
import json

with urlopen("https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json") as response:
    counties = json.load(response)

counties["features"][0]

Each `feature.id` is a FIPS code (Federal Information Processing Standards), which is a number that uniquely identifies a geographic area.

Next, we load unemployment data by county, also indexed by FIPS code.

In [ ]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/plotly/datasets/master/fips-unemp-16.csv",
    dtype={"fips": str},
)
df.head()

We are ready for plotting. We'll use Plotly express, Plotly's high-level API for creating figures. As it was for geoplot, we can use a very convenient `choropleth()` function.

In [ ]:
import plotly.express as px

# option 1
fig = px.choropleth(
    df,
    geojson=counties,
    locations="fips",
    color="unemp",
    color_continuous_scale="Viridis",
    range_color=(0, 12),
    scope="usa",
    labels={"unemp": "unemployment rate"},
)

fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show()

In [ ]:
fig = px.choropleth(
    df,
    geojson=counties,
    locations="fips",
    color="unemp",
    color_continuous_scale="Viridis",
    range_color=(0, 12),
    scope="usa",
    labels={"unemp": "unemployment rate"},
)

fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})

# better legend
fig.update_layout(
    coloraxis_colorbar=dict(
        thicknessmode="pixels",
        thickness=10,
        lenmode="pixels",
        len=150,
        yanchor="top",
        y=0.8,
        ticks="outside",
        ticksuffix=" %",
        dtick=5,
    )
)

fig.show()

In [ ]:
# option 2
fig = px.choropleth_mapbox(
    df,
    geojson=counties,
    locations="fips",
    color="unemp",
    color_continuous_scale="Viridis",
    range_color=(0, 12),
    mapbox_style="carto-positron",
    zoom=3,
    center={"lat": 37.0902, "lon": -95.7129},
    opacity=0.5,
    labels={"unemp": "unemployment rate"},
)

fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show()

Especially if you work outside Jupyter notebooks, you can save the output plot as a standalone HTML file:

In [ ]:
fig.write_html("plotly_choropleth_unemp.html")

You can then embed the HTML map in any HTML document using an `iframe`:

`<iframe src="plotly_choropleth_unemp.html" title="Plotly choropleth map" style={{ border: "none", width: "800px", height: "300px" }}></iframe>`

# Folium

Folium is a Python wrapper of the `leaflet.js` JavaScript library, putting together the best of the two worlds: Python's data processing capabilities and JavaScript's interactive data visualization.

With Folium we can create a map with just one line of code (besides the `import`):

In [ ]:
# installation
# !pip install folium
# !conda install -y -c conda-forge folium
# !mamba install -y folium

In [ ]:
import folium
map = folium.Map(location=[47.6151, -122.3413], zoom_start=13)
map

As you see, by default Folium uses OpenStreetMap as tile provider.

The map can be saved to a standalone HTML file:

In [ ]:
map.save("folium_basic_map.html")

You can use several options for the background tile using the `tiles` parameter. For example, let's take a look at Paris with a `CartoDB Positron` or `CartoDB Voyager` tile:

In [ ]:
custoMap = folium.Map(location=[48.85, 2.35], tiles="cartodb positron", zoom_start=10)
custoMap

In [ ]:
custoMap = folium.Map(location=[48.85, 2.35], tiles="cartodb voyager", zoom_start=10)
custoMap

You can also pass a custom tileset to `tiles` with a URL template, such as "https://{s}.tiles.example.com/{z}/{x}/{y}.png". 

Pick one from here: https://leaflet-extras.github.io/leaflet-providers/preview/

In [ ]:
custoMap = folium.Map(
    location=[48.85, 2.35],
    tiles="https://{s}.tile.opentopomap.org/{z}/{x}/{y}.png",
    attr="OpenTopoMap",
    zoom_start=10,
)
custoMap

Adding markers to a Folium map is as easy as creating a Pandas dataframe storing their coordinates.

In [ ]:
# Make an empty map
m = folium.Map(location=[20, 0], zoom_start=2)
m

In [ ]:
# Make a data frame with dots to show on the map
data = pd.DataFrame({
    'lon': [-58, 2, 145, -73.57, 36.82],
    'lat': [-34, 49, -38, 45.52, -1.29],
    'name': ['Buenos Aires', 'Paris', 'Melbourne', 'Montreal', 'Nairobi'],
    'value': [10, 12, 40, 43, 100]
})

data

In [ ]:
# add marker one by one on the map
for i in range(0, len(data)):
    folium.Marker(
        location=[data.iloc[i]["lat"], data.iloc[i]["lon"]],
        popup=f"{data.iloc[i]['name']}\n{data.iloc[i]['value']}",
    ).add_to(m)

# Show the map again
m

Time to create a choropleth map of the US unemployment rate with Folium. First, we grab the data: note that this time we only need a valid path / URL for the geoJSON with geometry and we do not need to read it in. We do need to read the US unemployment data though.

In [ ]:
# geometry data
url = "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data"
state_geo = f"{url}/us-states.json"
# unemployment data
state_unemployment = f"{url}/US_Unemployment_Oct2012.csv"
state_data = pd.read_csv(state_unemployment)
state_data.head()

Next, we initialize a map setting tile and location:

In [ ]:
m = folium.Map(location=[40, -95], zoom_start=4)
m

Finally, we create a `folium.Choropleth` object and we add it to our map:

In [ ]:
folium.Choropleth(
    geo_data=state_geo,
    name="choropleth",
    data=state_data,
    columns=["State", "Unemployment"],
    key_on="feature.id",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.1,
    legend_name="Unemployment Rate (%)",
).add_to(m)

folium.LayerControl().add_to(m)

m

---

(partially abridged from [The Python Graph Gallery](https://www.python-graph-gallery.com/choropleth-map/), [Plotly doc](https://plotly.com/python/choropleth-maps/), [Folium doc](https://python-visualization.github.io/folium/quickstart.html))